In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
# # Load the recycled dataset
train_data = np.load('./dataset/recycled_32_train.npz')
test_data = np.load('./dataset/recycled_32_test.npz')
# train_data = np.load("/content/drive/MyDrive/hAQathon/dataset/recycled_32_train.npz")
# test_data = np.load("/content/drive/MyDrive/hAQathon/dataset/recycled_32_test.npz")

x_train = train_data['x']
y_train = train_data['y']
x_test = test_data['x']
y_test = test_data['y']

# Preprocess the data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reshape the data
x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)

x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)

In [3]:
# Random circuit parameters
n_epochs = 50   # Number of optimization epochs
n_layers = 1    # Number of random layers
n_train = 50    # Size of the train dataset
n_test = 30     # Size of the test dataset
n_qubits = 16

dev = qml.device("default.qubit", wires=n_qubits)
rand_params = np.random.uniform(high=2 * np.pi, size=(n_layers, n_qubits))


@qml.qnode(dev, interface="autograd")
def circuit(phi):
    # Encoding of 4 classical input values
    for j in range(n_qubits):
        qml.RY(np.pi * phi[j], wires=j)

    # Random quantum circuit
    RandomLayers(rand_params, wires=list(range(n_qubits)))

    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(n_qubits)]

In [4]:
def quanv(image):
    """Convolves the input image with many applications of the same quantum circuit."""
    out = np.zeros((16, 16, n_qubits))

    # Loop over the coordinates of the top-left pixel of 2X2 squares
    for j in range(0, 32, int(n_qubits**(0.5))):
        for k in range(0, 32, int(n_qubits**(0.5))):
            # Process a squared 2x2 region of the image with a quantum circuit
            q_results = circuit(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j, k + 2, 0],
                    image[j, k + 3, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0],
                    image[j + 1, k + 2, 0],
                    image[j + 1, k + 3, 0],
                    image[j + 2, k, 0],
                    image[j + 2, k + 1, 0],
                    image[j + 2, k + 2, 0],
                    image[j + 2, k + 3, 0],
                    image[j + 3, k, 0],
                    image[j + 3, k + 1, 0],
                    image[j + 3, k + 2, 0],
                    image[j + 3, k + 3, 0],
                ]
            )
            # Assign expectation values to different channels of the output pixel (j/2, k/2)
            for c in range(n_qubits):
                out[j // 2, k // 2, c] = q_results[c]
    return out

In [5]:
train_size = 10000
test_size = 1500

In [6]:
SAVE_PATH = "quanvolution/"
PREPROCESS = True

if PREPROCESS:
    q_train_images = []
    print("Quantum pre-processing of train images:")
    for idx, img in enumerate(x_train):
        print("{}/{}        ".format(idx + 1, 10000), end="\r")
        q_train_images.append(quanv(img))
    q_train_images = np.asarray(q_train_images)

    q_test_images = []
    print("\nQuantum pre-processing of test images:")
    for idx, img in enumerate(x_test):
        print("{}/{}        ".format(idx + 1, 1500), end="\r")
        q_test_images.append(quanv(img))
    q_test_images = np.asarray(q_test_images)

    # Save pre-processed images
    np.save(SAVE_PATH + "q_train_images.npy", q_train_images)
    np.save(SAVE_PATH + "q_test_images.npy", q_test_images)


# Load pre-processed images
q_train_images = np.load(SAVE_PATH + "q_train_images.npy")
q_test_images = np.load(SAVE_PATH + "q_test_images.npy")

Quantum pre-processing of train images:
10000/10000        
Quantum pre-processing of test images:


In [8]:
# SAVE_PATH = "quanvolution/"
# np.save(SAVE_PATH + "q_train_images.npy", q_train_images)
# np.save(SAVE_PATH + "q_test_images.npy", q_test_images)

In [7]:
# Compile the model
def MyModel():
    """Initializes and returns a custom Keras model
    which is ready to be trained."""
    model = keras.models.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(1000, activation="softmax")
    ])

    model.compile(
        optimizer='adam',
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [8]:
# Step 2: Training
q_model = MyModel()

q_history = q_model.fit(
    q_train_images,
    y_train,
    validation_data=(q_test_images, y_test),
    batch_size=32,
    epochs=n_epochs,
    verbose=2,
)

Epoch 1/50
313/313 - 2s - loss: 1.3605 - accuracy: 0.4957 - val_loss: 1.1601 - val_accuracy: 0.5313 - 2s/epoch - 6ms/step
Epoch 2/50
313/313 - 2s - loss: 1.1005 - accuracy: 0.5714 - val_loss: 1.1187 - val_accuracy: 0.5280 - 2s/epoch - 5ms/step
Epoch 3/50
313/313 - 2s - loss: 1.0507 - accuracy: 0.5912 - val_loss: 1.0908 - val_accuracy: 0.5533 - 2s/epoch - 5ms/step
Epoch 4/50
313/313 - 2s - loss: 1.0193 - accuracy: 0.6071 - val_loss: 1.0621 - val_accuracy: 0.5780 - 2s/epoch - 5ms/step
Epoch 5/50
313/313 - 2s - loss: 0.9961 - accuracy: 0.6143 - val_loss: 1.0398 - val_accuracy: 0.5713 - 2s/epoch - 5ms/step
Epoch 6/50
313/313 - 2s - loss: 0.9710 - accuracy: 0.6225 - val_loss: 1.0268 - val_accuracy: 0.5687 - 2s/epoch - 5ms/step
Epoch 7/50
313/313 - 2s - loss: 0.9561 - accuracy: 0.6330 - val_loss: 1.0155 - val_accuracy: 0.5927 - 2s/epoch - 5ms/step
Epoch 8/50
313/313 - 2s - loss: 0.9433 - accuracy: 0.6302 - val_loss: 1.0127 - val_accuracy: 0.5667 - 2s/epoch - 5ms/step
Epoch 9/50
313/313 - 2s 

In [9]:
# Step 3: Evaluation
loss, accuracy = q_model.evaluate(q_test_images, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

47/47 [==============================] - 0s 2ms/step - loss: 0.9000 - accuracy: 0.6307
Loss: 0.900017261505127
Accuracy: 0.6306666731834412
